In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

In [2]:
df=pd.read_csv(r'G:\GitHub\Kaggle Tokyo Stock Exchange\jpx-tokyo-stock-exchange-prediction\train_files\stock_prices.csv')
df

,RowId,Date,SecuritiesCode,Open,High,Low,Close,Volume,AdjustmentFactor,ExpectedDividend,SupervisionFlag,Target
0,20170104_1301,2017-01-04,1301,2734.0,2755.0,2730.0,2742.0,31400,1.0,NaN,False,0.000730
1,20170104_1332,2017-01-04,1332,568.0,576.0,563.0,571.0,2798500,1.0,NaN,False,0.012324
2,20170104_1333,2017-01-04,1333,3150.0,3210.0,3140.0,3210.0,270800,1.0,NaN,False,0.006154
3,20170104_1376,2017-01-04,1376,1510.0,1550.0,1510.0,1550.0,11300,1.0,NaN,False,0.011053
4,20170104_1377,2017-01-04,1377,3270.0,3350.0,3270.0,3330.0,150800,1.0,NaN,False,0.003026
...,...,...,...,...,...,...,...,...,...,...,...,...
2332526,20211203_9990,2021-12-03,9990,514.0,528.0,513.0,528.0,44200,1.0,NaN,False,0.034816
2332527,20211203_9991,2021-12-03,9991,782.0,794.0,782.0,794.0,35900,1.0,NaN,False,0.025478
2332528,20211203_9993,2021-12-03,9993,1690.0,1690.0,1645.0,1645.0,7200,1.0,NaN,False,-0.004302
2332529,20211203_9994,2021-12-03,9994,2388.0,2396.0,2380.0,2389.0,6500,1.0,NaN,False,0.009098


In [3]:
import datetime

def str_to_datetime(s):
  split = s.split('-')
  year, month, day = int(split[0]), int(split[1]), int(split[2])
  return datetime.datetime(year=year, month=month, day=day)

datetime_object = str_to_datetime('1986-03-19')
datetime_object

datetime.datetime(1986, 3, 19, 0, 0)

In [4]:
df['Date']=df['Date'].apply(str_to_datetime)
df['Date']

0         2017-01-04
1         2017-01-04
2         2017-01-04
3         2017-01-04
4         2017-01-04
             ...    
2332526   2021-12-03
2332527   2021-12-03
2332528   2021-12-03
2332529   2021-12-03
2332530   2021-12-03
Name: Date, Length: 2332531, dtype: datetime64[ns]

In [5]:
df['Date'].max()

Timestamp('2021-12-03 00:00:00')

In [6]:
import os
from decimal import ROUND_HALF_UP, Decimal

In [7]:
#Generating AdjustedClose price
#We will generate AdjustedClose using AdjustmentFactor value. This should reduce historical price gap caused by split/reverse-split.
def adjust_price(price):
    """
    Args:
        price (pd.DataFrame)  : pd.DataFrame include stock_price
    Returns:
        price DataFrame (pd.DataFrame): stock_price with generated AdjustedClose
    """
    # transform Date column into datetime (done!)
    # price.loc[: ,"Date"] = pd.to_datetime(price.loc[: ,"Date"], format="%Y-%m-%d")

    def generate_adjusted_close(df):
        """
        Args:
            df (pd.DataFrame)  : stock_price for a single SecuritiesCode
        Returns:
            df (pd.DataFrame): stock_price with AdjustedClose for a single SecuritiesCode
        """
        # sort data to generate CumulativeAdjustmentFactor
        df = df.sort_values("Date", ascending=False)
        # generate CumulativeAdjustmentFactor
        df.loc[:, "CumulativeAdjustmentFactor"] = df["AdjustmentFactor"].cumprod()
        # generate AdjustedClose
        df.loc[:, "AdjustedClose"] = (
            df["CumulativeAdjustmentFactor"] * df["Close"]
        ).map(lambda x: float(
            Decimal(str(x)).quantize(Decimal('0.1'), rounding=ROUND_HALF_UP)
        ))
        # reverse order
        df = df.sort_values("Date")
        # to fill AdjustedClose, replace 0 into np.nan
        df.loc[df["AdjustedClose"] == 0, "AdjustedClose"] = np.nan
        # forward fill AdjustedClose
        df.loc[:, "AdjustedClose"] = df.loc[:, "AdjustedClose"].ffill()
        return df

    # generate AdjustedClose
    price = price.sort_values(["SecuritiesCode", "Date"])
    price = price.groupby("SecuritiesCode").apply(generate_adjusted_close).reset_index(drop=True)

    # price.set_index("Date", inplace=True)
    return price

In [8]:
df_price= adjust_price(df)
df_price

,RowId,Date,SecuritiesCode,Open,High,Low,Close,Volume,AdjustmentFactor,ExpectedDividend,SupervisionFlag,Target,CumulativeAdjustmentFactor,AdjustedClose
0,20170104_1301,2017-01-04,1301,2734.0,2755.0,2730.0,2742.0,31400,1.0,NaN,False,0.000730,1.0,2742.0
1,20170105_1301,2017-01-05,1301,2743.0,2747.0,2735.0,2738.0,17900,1.0,NaN,False,0.002920,1.0,2738.0
2,20170106_1301,2017-01-06,1301,2734.0,2744.0,2720.0,2740.0,19900,1.0,NaN,False,-0.001092,1.0,2740.0
3,20170110_1301,2017-01-10,1301,2745.0,2754.0,2735.0,2748.0,24200,1.0,NaN,False,-0.005100,1.0,2748.0
4,20170111_1301,2017-01-11,1301,2748.0,2752.0,2737.0,2745.0,9300,1.0,NaN,False,-0.003295,1.0,2745.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2332526,20211129_9997,2021-11-29,9997,678.0,679.0,665.0,668.0,320800,1.0,NaN,False,0.026987,1.0,668.0
2332527,20211130_9997,2021-11-30,9997,670.0,689.0,667.0,667.0,296300,1.0,NaN,False,-0.001460,1.0,667.0
2332528,20211201_9997,2021-12-01,9997,661.0,688.0,660.0,685.0,339100,1.0,NaN,False,0.017544,1.0,685.0
2332529,20211202_9997,2021-12-02,9997,681.0,692.0,680.0,684.0,342900,1.0,NaN,False,0.014368,1.0,684.0


# Generating Features

It's worth to examine the impact of each company financial indicators on its stock price such as Earning per share(EPS),   total sales, total purchase of the whole market on a particular day. These indicators might be of investor's interest on the buy/sell decision and could possibly be good predicative features. All of this info is derived from other excel files. 

In [9]:
financials=pd.read_csv(r'G:\GitHub\Kaggle Tokyo Stock Exchange\jpx-tokyo-stock-exchange-prediction\train_files\financials.csv')
financials.loc[: ,"Date"] = pd.to_datetime(financials.loc[: ,"Date"], format="%Y-%m-%d")
financials.head(20)

C:\Users\Tram\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3331: DtypeWarning: Columns (14,15,16,17,18,19,42,44) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,DisclosureNumber,DateCode,Date,SecuritiesCode,DisclosedDate,DisclosedTime,DisclosedUnixTime,TypeOfDocument,CurrentPeriodEndDate,TypeOfCurrentPeriod,...,ForecastEarningsPerShare,ApplyingOfSpecificAccountingOfTheQuarterlyFinancialStatements,MaterialChangesInSubsidiaries,ChangesBasedOnRevisionsOfAccountingStandard,ChangesOtherThanOnesBasedOnRevisionsOfAccountingStandard,ChangesInAccountingEstimates,RetrospectiveRestatement,NumberOfIssuedAndOutstandingSharesAtTheEndOfFiscalYearIncludingTreasuryStock,NumberOfTreasuryStockAtTheEndOfFiscalYear,AverageNumberOfShares
0,2.016121e+13,20170104_2753,2017-01-04,2753.0,2017-01-04,07:30:00,1.483483e+09,3QFinancialStatements_Consolidated_JP,2016-12-31,3Q,...,319.76,NaN,False,True,False,False,False,6848800.0,－,6848800.0
1,2.017010e+13,20170104_3353,2017-01-04,3353.0,2017-01-04,15:00:00,1.483510e+09,3QFinancialStatements_Consolidated_JP,2016-11-30,3Q,...,485.36,NaN,False,True,False,False,False,2035000.0,118917,1916083.0
2,2.016123e+13,20170104_4575,2017-01-04,4575.0,2017-01-04,12:00:00,1.483499e+09,ForecastRevision,2016-12-31,2Q,...,-93.11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2.017010e+13,20170105_2659,2017-01-05,2659.0,2017-01-05,15:00:00,1.483596e+09,3QFinancialStatements_Consolidated_JP,2016-11-30,3Q,...,285.05,NaN,False,True,False,False,False,31981654.0,18257,31963405.0
4,2.017011e+13,20170105_3050,2017-01-05,3050.0,2017-01-05,15:30:00,1.483598e+09,ForecastRevision,2017-02-28,FY,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2.016123e+13,20170105_3139,2017-01-05,3139.0,2017-01-05,15:30:00,1.483598e+09,ForecastRevision,2016-11-30,FY,...,173.43,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2.017010e+13,20170105_3282,2017-01-05,3282.0,2017-01-05,15:30:00,1.483598e+09,ForecastRevision_REIT,2017-01-31,FY,...,4558,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2.017011e+13,20170105_3344,2017-01-05,3344.0,2017-01-05,15:00:00,1.483596e+09,3QFinancialStatements_Consolidated_JP,2016-11-30,3Q,...,47.7,NaN,False,True,False,False,False,5579184.0,1372,5578257.0
8,2.017011e+13,20170105_4579,2017-01-05,4579.0,2017-01-05,18:30:00,1.483609e+09,ForecastRevision,2016-12-31,FY,...,-47.37,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2.017010e+13,20170105_7453,2017-01-05,7453.0,2017-01-05,15:00:00,1.483596e+09,3QFinancialStatements_Consolidated_JP,2016-11-30,3Q,...,913.28,NaN,False,True,True,False,False,28078000.0,1702372,26527712.0


In [10]:
financials.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 92956 entries, 0 to 92955
Data columns (total 45 columns):
 #   Column                                                                        Non-Null Count  Dtype         
---  ------                                                                        --------------  -----         
 0   DisclosureNumber                                                              92954 non-null  float64       
 1   DateCode                                                                      92954 non-null  object        
 2   Date                                                                          92956 non-null  datetime64[ns]
 3   SecuritiesCode                                                                92954 non-null  float64       
 4   DisclosedDate                                                                 92954 non-null  object        
 5   DisclosedTime                                                                 92954 non-

In [11]:
financials.groupby(['SecuritiesCode', 'Date'])['ForecastEarningsPerShare'].count()

SecuritiesCode  Date      
1301.0          2017-02-10    1
                2017-02-17    0
                2017-05-11    1
                2017-08-04    1
                2017-11-06    1
                             ..
9997.0          2021-01-28    1
                2021-01-29    1
                2021-05-13    1
                2021-07-30    1
                2021-10-29    1
Name: ForecastEarningsPerShare, Length: 87290, dtype: int64

In [12]:
financials_df=financials[['SecuritiesCode','Date','ForecastEarningsPerShare']]

In [13]:
df_price[df_price['SecuritiesCode'] == 7952]

,RowId,Date,SecuritiesCode,Open,High,Low,Close,Volume,AdjustmentFactor,ExpectedDividend,SupervisionFlag,Target,CumulativeAdjustmentFactor,AdjustedClose
1742351,20170104_7952,2017-01-04,7952,2310.0,2357.0,2310.0,2344.0,11900,1.0,NaN,False,0.011548,1.0,2344.0
1742352,20170105_7952,2017-01-05,7952,2355.0,2355.0,2321.0,2338.0,7900,1.0,NaN,False,-0.007611,1.0,2338.0
1742353,20170106_7952,2017-01-06,7952,2319.0,2369.0,2302.0,2365.0,12200,1.0,NaN,False,0.000000,1.0,2365.0
1742354,20170110_7952,2017-01-10,7952,2339.0,2350.0,2321.0,2347.0,8400,1.0,NaN,False,-0.013634,1.0,2347.0
1742355,20170111_7952,2017-01-11,7952,2347.0,2347.0,2318.0,2347.0,5100,1.0,NaN,False,-0.005616,1.0,2347.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1743548,20211129_7952,2021-11-29,7952,3200.0,3280.0,3200.0,3205.0,22100,1.0,NaN,False,0.001558,1.0,3205.0
1743549,20211130_7952,2021-11-30,7952,3235.0,3315.0,3190.0,3210.0,17800,1.0,NaN,False,-0.010886,1.0,3210.0
1743550,20211201_7952,2021-12-01,7952,3170.0,3240.0,3120.0,3215.0,29100,1.0,NaN,False,0.029874,1.0,3215.0
1743551,20211202_7952,2021-12-02,7952,3195.0,3260.0,3165.0,3180.0,16000,1.0,NaN,False,0.004580,1.0,3180.0


In [14]:
merged_df = df_price.merge(financials_df, on=['SecuritiesCode','Date'], how='left')
merged_df[['RowId', 'SecuritiesCode', 'Open', 'High', 'Low', 'Close', 'Volume',
       'AdjustmentFactor', 'ExpectedDividend', 'SupervisionFlag', 'Target',
       'CumulativeAdjustmentFactor', 'AdjustedClose','ForecastEarningsPerShare']]

,RowId,SecuritiesCode,Open,High,Low,Close,Volume,AdjustmentFactor,ExpectedDividend,SupervisionFlag,Target,CumulativeAdjustmentFactor,AdjustedClose,ForecastEarningsPerShare
0,20170104_1301,1301,2734.0,2755.0,2730.0,2742.0,31400,1.0,NaN,False,0.000730,1.0,2742.0,NaN
1,20170105_1301,1301,2743.0,2747.0,2735.0,2738.0,17900,1.0,NaN,False,0.002920,1.0,2738.0,NaN
2,20170106_1301,1301,2734.0,2744.0,2720.0,2740.0,19900,1.0,NaN,False,-0.001092,1.0,2740.0,NaN
3,20170110_1301,1301,2745.0,2754.0,2735.0,2748.0,24200,1.0,NaN,False,-0.005100,1.0,2748.0,NaN
4,20170111_1301,1301,2748.0,2752.0,2737.0,2745.0,9300,1.0,NaN,False,-0.003295,1.0,2745.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2335596,20211129_9997,9997,678.0,679.0,665.0,668.0,320800,1.0,NaN,False,0.026987,1.0,668.0,NaN
2335597,20211130_9997,9997,670.0,689.0,667.0,667.0,296300,1.0,NaN,False,-0.001460,1.0,667.0,NaN
2335598,20211201_9997,9997,661.0,688.0,660.0,685.0,339100,1.0,NaN,False,0.017544,1.0,685.0,NaN
2335599,20211202_9997,9997,681.0,692.0,680.0,684.0,342900,1.0,NaN,False,0.014368,1.0,684.0,NaN


In [15]:
merged_df

,RowId,Date,SecuritiesCode,Open,High,Low,Close,Volume,AdjustmentFactor,ExpectedDividend,SupervisionFlag,Target,CumulativeAdjustmentFactor,AdjustedClose,ForecastEarningsPerShare
0,20170104_1301,2017-01-04,1301,2734.0,2755.0,2730.0,2742.0,31400,1.0,NaN,False,0.000730,1.0,2742.0,NaN
1,20170105_1301,2017-01-05,1301,2743.0,2747.0,2735.0,2738.0,17900,1.0,NaN,False,0.002920,1.0,2738.0,NaN
2,20170106_1301,2017-01-06,1301,2734.0,2744.0,2720.0,2740.0,19900,1.0,NaN,False,-0.001092,1.0,2740.0,NaN
3,20170110_1301,2017-01-10,1301,2745.0,2754.0,2735.0,2748.0,24200,1.0,NaN,False,-0.005100,1.0,2748.0,NaN
4,20170111_1301,2017-01-11,1301,2748.0,2752.0,2737.0,2745.0,9300,1.0,NaN,False,-0.003295,1.0,2745.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2335596,20211129_9997,2021-11-29,9997,678.0,679.0,665.0,668.0,320800,1.0,NaN,False,0.026987,1.0,668.0,NaN
2335597,20211130_9997,2021-11-30,9997,670.0,689.0,667.0,667.0,296300,1.0,NaN,False,-0.001460,1.0,667.0,NaN
2335598,20211201_9997,2021-12-01,9997,661.0,688.0,660.0,685.0,339100,1.0,NaN,False,0.017544,1.0,685.0,NaN
2335599,20211202_9997,2021-12-02,9997,681.0,692.0,680.0,684.0,342900,1.0,NaN,False,0.014368,1.0,684.0,NaN


## Earning Per Share

In [16]:
# There are Null values in ForecastEarningsPerShare column due to mismatch in Date between 2 previous dataframes. Therefore, we have to fill in these null values with the values of previous disclosed days.
def fillin_null(code, EPS):
    new_EPS = EPS.copy()
    for i in range(1, len(code)):
        if pd.isnull(new_EPS[i]) and code[i] == code[i-1]:
            new_EPS[i] = new_EPS[i-1]
        else:
            new_EPS[i]
    return new_EPS
merged_df['EPS']=fillin_null(merged_df['SecuritiesCode'], merged_df['ForecastEarningsPerShare'])


In [17]:
# Check if the EPS is accurately allocated
merged_df[merged_df['SecuritiesCode'] == 7952].head(60)

,RowId,Date,SecuritiesCode,Open,High,Low,Close,Volume,AdjustmentFactor,ExpectedDividend,SupervisionFlag,Target,CumulativeAdjustmentFactor,AdjustedClose,ForecastEarningsPerShare,EPS
1744780,20170104_7952,2017-01-04,7952,2310.0,2357.0,2310.0,2344.0,11900,1.0,NaN,False,0.011548,1.0,2344.0,NaN,NaN
1744781,20170105_7952,2017-01-05,7952,2355.0,2355.0,2321.0,2338.0,7900,1.0,NaN,False,-0.007611,1.0,2338.0,NaN,NaN
1744782,20170106_7952,2017-01-06,7952,2319.0,2369.0,2302.0,2365.0,12200,1.0,NaN,False,0.000000,1.0,2365.0,NaN,NaN
1744783,20170110_7952,2017-01-10,7952,2339.0,2350.0,2321.0,2347.0,8400,1.0,NaN,False,-0.013634,1.0,2347.0,NaN,NaN
1744784,20170111_7952,2017-01-11,7952,2347.0,2347.0,2318.0,2347.0,5100,1.0,NaN,False,-0.005616,1.0,2347.0,NaN,NaN
1744785,20170112_7952,2017-01-12,7952,2331.0,2341.0,2230.0,2315.0,26700,1.0,NaN,False,-0.007385,1.0,2315.0,NaN,NaN
1744786,20170113_7952,2017-01-13,7952,2271.0,2309.0,2271.0,2302.0,12800,1.0,NaN,False,0.004376,1.0,2302.0,NaN,NaN
1744787,20170116_7952,2017-01-16,7952,2291.0,2291.0,2261.0,2285.0,5800,1.0,NaN,False,-0.006972,1.0,2285.0,NaN,NaN
1744788,20170117_7952,2017-01-17,7952,2300.0,2300.0,2252.0,2295.0,11000,1.0,NaN,False,-0.003072,1.0,2295.0,NaN,NaN
1744789,20170118_7952,2017-01-18,7952,2275.0,2283.0,2185.0,2279.0,15400,1.0,NaN,False,-0.001320,1.0,2279.0,NaN,NaN


In [18]:
# Check with the data in original dataset.
financials_df[financials_df['SecuritiesCode'] == 7952]

,SecuritiesCode,Date,ForecastEarningsPerShare
2725,7952.0,2017-02-09,159.14
7232,7952.0,2017-05-11,186.59
12967,7952.0,2017-08-10,186.59
16905,7952.0,2017-11-09,186.59
21080,7952.0,2018-02-08,189.52
25549,7952.0,2018-05-10,245.74
30716,7952.0,2018-08-09,245.74
34854,7952.0,2018-11-08,245.92
38892,7952.0,2019-02-07,245.92
43111,7952.0,2019-05-09,221.78


In [19]:
merged_df = merged_df.drop(['ForecastEarningsPerShare'], axis=1)
merged_df.columns=[['RowId','Date', 'SecuritiesCode', 'Open', 'High', 'Low', 'Close', 'Volume',
       'AdjustmentFactor', 'ExpectedDividend', 'SupervisionFlag', 'Target',
       'CumulativeAdjustmentFactor', 'AdjustedClose', 'ForecastEarningsPerShare']]

In [20]:
merged_df['ForecastEarningsPerShare'] = pd.to_numeric(merged_df[['ForecastEarningsPerShare']].squeeze(), errors='coerce')
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2335601 entries, 0 to 2335600
Data columns (total 15 columns):
 #   Column                         Dtype         
---  ------                         -----         
 0   (RowId,)                       object        
 1   (Date,)                        datetime64[ns]
 2   (SecuritiesCode,)              int64         
 3   (Open,)                        float64       
 4   (High,)                        float64       
 5   (Low,)                         float64       
 6   (Close,)                       float64       
 7   (Volume,)                      int64         
 8   (AdjustmentFactor,)            float64       
 9   (ExpectedDividend,)            float64       
 10  (SupervisionFlag,)             bool          
 11  (Target,)                      float64       
 12  (CumulativeAdjustmentFactor,)  float64       
 13  (AdjustedClose,)               float64       
 14  (ForecastEarningsPerShare,)    float64       
dtypes: bool(1), dat

## Total Sales, Total Purchase 

In [21]:
Trade=pd.read_csv(r'G:\GitHub\Kaggle Tokyo Stock Exchange\jpx-tokyo-stock-exchange-prediction\train_files\trades.csv')
Trade.loc[: ,"Date"] = pd.to_datetime(financials.loc[: ,"Date"], format="%Y-%m-%d")
Trade

,Date,StartDate,EndDate,Section,TotalSales,TotalPurchases,TotalTotal,TotalBalance,ProprietarySales,ProprietaryPurchases,...,CityBKsRegionalBKsEtcTotal,CityBKsRegionalBKsEtcBalance,TrustBanksSales,TrustBanksPurchases,TrustBanksTotal,TrustBanksBalance,OtherFinancialInstitutionsSales,OtherFinancialInstitutionsPurchases,OtherFinancialInstitutionsTotal,OtherFinancialInstitutionsBalance
0,2017-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2017-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2017-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2017-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2017-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1707,2017-02-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1708,2017-02-03,2021-11-22,2021-11-26,Growth Market (Mothers/JASDAQ),1.143466e+09,1.143923e+09,2.287389e+09,456677.0,3.663919e+07,3.496068e+07,...,396230.0,-275608.0,6696755.0,6886122.0,13582877.0,189367.0,234653.0,298525.0,533178.0,63872.0
1709,2017-02-03,2021-11-22,2021-11-26,Prime Market (First Section),1.138343e+10,1.137621e+10,2.275964e+10,-7214179.0,1.499660e+09,1.230944e+09,...,35957940.0,-17510292.0,254580089.0,261919512.0,516499601.0,7339423.0,11959898.0,16368287.0,28328185.0,4408389.0
1710,2017-02-03,2021-11-22,2021-11-26,Standard Market (Second Section),1.069969e+08,1.075036e+08,2.145004e+08,506702.0,2.811025e+06,3.273163e+06,...,42127.0,-42127.0,438928.0,243817.0,682745.0,-195111.0,60291.0,6985.0,67276.0,-53306.0


In [22]:
Trade[['StartDate','EndDate']].value_counts()

StartDate   EndDate   
2017-01-04  2017-01-06    3
2020-05-18  2020-05-22    3
2020-02-17  2020-02-21    3
2020-02-25  2020-02-28    3
2020-03-02  2020-03-06    3
                         ..
2018-09-25  2018-09-28    3
2018-10-01  2018-10-05    3
2018-10-09  2018-10-12    3
2018-10-15  2018-10-19    3
2021-11-22  2021-11-26    3
Length: 255, dtype: int64

Total Sales and Total Purchase of Each period within start and end date has been modified 3 times. Let's find out the average Total Sales and Total Purchase.


In [23]:
Trade['TotalSales'] = Trade.groupby('StartDate')['TotalSales'].transform('mean')
Trade

,Date,StartDate,EndDate,Section,TotalSales,TotalPurchases,TotalTotal,TotalBalance,ProprietarySales,ProprietaryPurchases,...,CityBKsRegionalBKsEtcTotal,CityBKsRegionalBKsEtcBalance,TrustBanksSales,TrustBanksPurchases,TrustBanksTotal,TrustBanksBalance,OtherFinancialInstitutionsSales,OtherFinancialInstitutionsPurchases,OtherFinancialInstitutionsTotal,OtherFinancialInstitutionsBalance
0,2017-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2017-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2017-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2017-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2017-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1707,2017-02-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1708,2017-02-03,2021-11-22,2021-11-26,Growth Market (Mothers/JASDAQ),4.211296e+09,1.143923e+09,2.287389e+09,456677.0,3.663919e+07,3.496068e+07,...,396230.0,-275608.0,6696755.0,6886122.0,13582877.0,189367.0,234653.0,298525.0,533178.0,63872.0
1709,2017-02-03,2021-11-22,2021-11-26,Prime Market (First Section),4.211296e+09,1.137621e+10,2.275964e+10,-7214179.0,1.499660e+09,1.230944e+09,...,35957940.0,-17510292.0,254580089.0,261919512.0,516499601.0,7339423.0,11959898.0,16368287.0,28328185.0,4408389.0
1710,2017-02-03,2021-11-22,2021-11-26,Standard Market (Second Section),4.211296e+09,1.075036e+08,2.145004e+08,506702.0,2.811025e+06,3.273163e+06,...,42127.0,-42127.0,438928.0,243817.0,682745.0,-195111.0,60291.0,6985.0,67276.0,-53306.0


In [24]:
Trade['TotalPurchases'] = Trade.groupby('StartDate')['TotalPurchases'].transform('mean')
Trade

,Date,StartDate,EndDate,Section,TotalSales,TotalPurchases,TotalTotal,TotalBalance,ProprietarySales,ProprietaryPurchases,...,CityBKsRegionalBKsEtcTotal,CityBKsRegionalBKsEtcBalance,TrustBanksSales,TrustBanksPurchases,TrustBanksTotal,TrustBanksBalance,OtherFinancialInstitutionsSales,OtherFinancialInstitutionsPurchases,OtherFinancialInstitutionsTotal,OtherFinancialInstitutionsBalance
0,2017-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2017-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2017-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2017-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2017-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1707,2017-02-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1708,2017-02-03,2021-11-22,2021-11-26,Growth Market (Mothers/JASDAQ),4.211296e+09,4.209213e+09,2.287389e+09,456677.0,3.663919e+07,3.496068e+07,...,396230.0,-275608.0,6696755.0,6886122.0,13582877.0,189367.0,234653.0,298525.0,533178.0,63872.0
1709,2017-02-03,2021-11-22,2021-11-26,Prime Market (First Section),4.211296e+09,4.209213e+09,2.275964e+10,-7214179.0,1.499660e+09,1.230944e+09,...,35957940.0,-17510292.0,254580089.0,261919512.0,516499601.0,7339423.0,11959898.0,16368287.0,28328185.0,4408389.0
1710,2017-02-03,2021-11-22,2021-11-26,Standard Market (Second Section),4.211296e+09,4.209213e+09,2.145004e+08,506702.0,2.811025e+06,3.273163e+06,...,42127.0,-42127.0,438928.0,243817.0,682745.0,-195111.0,60291.0,6985.0,67276.0,-53306.0


In [25]:
Trade_df = pd.concat([Trade['StartDate'],Trade['EndDate'],Trade['TotalSales'],Trade['TotalPurchases']], axis=1)
Trade_df.columns= [['Date','EndDate','TotalSales','TotalPurchases']] # change name of StartDate to Date because of merging purpose later.
Trade_df

,Date,EndDate,TotalSales,TotalPurchases
0,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN
...,...,...,...,...
1707,NaN,NaN,NaN,NaN
1708,2021-11-22,2021-11-26,4.211296e+09,4.209213e+09
1709,2021-11-22,2021-11-26,4.211296e+09,4.209213e+09
1710,2021-11-22,2021-11-26,4.211296e+09,4.209213e+09


In [26]:
# Remove duplicate rows 
Trade_df=Trade_df.drop_duplicates()
Trade_df

,Date,EndDate,TotalSales,TotalPurchases
0,NaN,NaN,NaN,NaN
6,2017-01-04,2017-01-06,3.052787e+09,3.054814e+09
12,2017-01-10,2017-01-13,3.710609e+09,3.709987e+09
19,2017-01-16,2017-01-20,4.191575e+09,4.190943e+09
26,2017-01-23,2017-01-27,4.681441e+09,4.681005e+09
...,...,...,...,...
1681,2021-10-25,2021-10-29,7.402267e+09,7.400393e+09
1688,2021-11-01,2021-11-05,4.956410e+09,4.950468e+09
1695,2021-11-08,2021-11-12,5.424703e+09,5.417969e+09
1701,2021-11-15,2021-11-19,5.706580e+09,5.704635e+09


In [27]:
Trade_df.columns

MultiIndex([(          'Date',),
            (       'EndDate',),
            (    'TotalSales',),
            ('TotalPurchases',)],
           )

In [28]:
# Convert multiIndex into single index
Trade_df.columns = Trade_df.columns.get_level_values(0)
Trade_df.columns

Index(['Date', 'EndDate', 'TotalSales', 'TotalPurchases'], dtype='object')

In [29]:
Trade_df['Date'] = Trade_df['Date'].apply(lambda x: x[0] if isinstance(x, tuple) else x)
Trade_df['EndDate'] = Trade_df['EndDate'].apply(lambda x: x[0] if isinstance(x, tuple) else x)

Trade_df['Date'] = pd.to_datetime(Trade_df['Date'])
Trade_df['EndDate'] = pd.to_datetime(Trade_df['EndDate'])


C:\Users\Tram\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\Tram\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\Tram\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in th

In [30]:
merged_df.columns = merged_df.columns.get_level_values(0)

In [31]:
merged_df.columns

Index(['RowId', 'Date', 'SecuritiesCode', 'Open', 'High', 'Low', 'Close',
       'Volume', 'AdjustmentFactor', 'ExpectedDividend', 'SupervisionFlag',
       'Target', 'CumulativeAdjustmentFactor', 'AdjustedClose',
       'ForecastEarningsPerShare'],
      dtype='object')

In [32]:
# Reset index of both DataFrames
merged_df = merged_df.reset_index()
Trade_df = Trade_df.reset_index()

# Merge based on 'Date' and 'EndDate'
merged_df = pd.merge(merged_df, Trade_df, on='Date', how='left')


In [33]:
merged_df


,index_x,RowId,Date,SecuritiesCode,Open,High,Low,Close,Volume,AdjustmentFactor,ExpectedDividend,SupervisionFlag,Target,CumulativeAdjustmentFactor,AdjustedClose,ForecastEarningsPerShare,index_y,EndDate,TotalSales,TotalPurchases
0,0,20170104_1301,2017-01-04,1301,2734.0,2755.0,2730.0,2742.0,31400,1.0,NaN,False,0.000730,1.0,2742.0,NaN,6.0,2017-01-06,3.052787e+09,3.054814e+09
1,1,20170105_1301,2017-01-05,1301,2743.0,2747.0,2735.0,2738.0,17900,1.0,NaN,False,0.002920,1.0,2738.0,NaN,NaN,NaT,NaN,NaN
2,2,20170106_1301,2017-01-06,1301,2734.0,2744.0,2720.0,2740.0,19900,1.0,NaN,False,-0.001092,1.0,2740.0,NaN,NaN,NaT,NaN,NaN
3,3,20170110_1301,2017-01-10,1301,2745.0,2754.0,2735.0,2748.0,24200,1.0,NaN,False,-0.005100,1.0,2748.0,NaN,12.0,2017-01-13,3.710609e+09,3.709987e+09
4,4,20170111_1301,2017-01-11,1301,2748.0,2752.0,2737.0,2745.0,9300,1.0,NaN,False,-0.003295,1.0,2745.0,NaN,NaN,NaT,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2335596,2335596,20211129_9997,2021-11-29,9997,678.0,679.0,665.0,668.0,320800,1.0,NaN,False,0.026987,1.0,668.0,129.3,NaN,NaT,NaN,NaN
2335597,2335597,20211130_9997,2021-11-30,9997,670.0,689.0,667.0,667.0,296300,1.0,NaN,False,-0.001460,1.0,667.0,129.3,NaN,NaT,NaN,NaN
2335598,2335598,20211201_9997,2021-12-01,9997,661.0,688.0,660.0,685.0,339100,1.0,NaN,False,0.017544,1.0,685.0,129.3,NaN,NaT,NaN,NaN
2335599,2335599,20211202_9997,2021-12-02,9997,681.0,692.0,680.0,684.0,342900,1.0,NaN,False,0.014368,1.0,684.0,129.3,NaN,NaT,NaN,NaN


In [34]:
merged_df=merged_df.drop(['index_x', 'RowId', 'Open',
       'High', 'Low', 'Close',  'AdjustmentFactor',
        'SupervisionFlag', 
       'CumulativeAdjustmentFactor', 
         'index_y'], axis = 1)
merged_df

,Date,SecuritiesCode,Volume,ExpectedDividend,Target,AdjustedClose,ForecastEarningsPerShare,EndDate,TotalSales,TotalPurchases
0,2017-01-04,1301,31400,NaN,0.000730,2742.0,NaN,2017-01-06,3.052787e+09,3.054814e+09
1,2017-01-05,1301,17900,NaN,0.002920,2738.0,NaN,NaT,NaN,NaN
2,2017-01-06,1301,19900,NaN,-0.001092,2740.0,NaN,NaT,NaN,NaN
3,2017-01-10,1301,24200,NaN,-0.005100,2748.0,NaN,2017-01-13,3.710609e+09,3.709987e+09
4,2017-01-11,1301,9300,NaN,-0.003295,2745.0,NaN,NaT,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
2335596,2021-11-29,9997,320800,NaN,0.026987,668.0,129.3,NaT,NaN,NaN
2335597,2021-11-30,9997,296300,NaN,-0.001460,667.0,129.3,NaT,NaN,NaN
2335598,2021-12-01,9997,339100,NaN,0.017544,685.0,129.3,NaT,NaN,NaN
2335599,2021-12-02,9997,342900,NaN,0.014368,684.0,129.3,NaT,NaN,NaN


In [35]:
# Fillin Nan in EndDate, TotalSales, TotalPurchases:
merged_df['EndDate'].fillna(method='ffill', inplace=True)
merged_df['TotalSales'].fillna(method='ffill', inplace=True)
merged_df['TotalPurchases'].fillna(method='ffill', inplace=True)

In [36]:
merged_df.set_index("Date", inplace=True)
merged_df.head(50)

,SecuritiesCode,Volume,ExpectedDividend,Target,AdjustedClose,ForecastEarningsPerShare,EndDate,TotalSales,TotalPurchases
Date,,,,,,,,,
2017-01-04,1301,31400,NaN,0.000730,2742.0,NaN,2017-01-06,3.052787e+09,3.054814e+09
2017-01-05,1301,17900,NaN,0.002920,2738.0,NaN,2017-01-06,3.052787e+09,3.054814e+09
2017-01-06,1301,19900,NaN,-0.001092,2740.0,NaN,2017-01-06,3.052787e+09,3.054814e+09
2017-01-10,1301,24200,NaN,-0.005100,2748.0,NaN,2017-01-13,3.710609e+09,3.709987e+09
2017-01-11,1301,9300,NaN,-0.003295,2745.0,NaN,2017-01-13,3.710609e+09,3.709987e+09
2017-01-12,1301,28700,NaN,-0.006613,2731.0,NaN,2017-01-13,3.710609e+09,3.709987e+09
2017-01-13,1301,19400,NaN,-0.006657,2722.0,NaN,2017-01-13,3.710609e+09,3.709987e+09
2017-01-16,1301,20100,NaN,0.002978,2704.0,NaN,2017-01-20,4.191575e+09,4.190943e+09
2017-01-17,1301,18400,NaN,0.001856,2686.0,NaN,2017-01-20,4.191575e+09,4.190943e+09


## Percentage change in price
When calculating the rate of price change in the last 2 week, last 1 month, or last 3 months, it is often treated as X business days later, such as 10 business days, 21 business days, or 63 business days later, as shown below, rather than X months later on the calendar to simplify handling in practical terms. In this case, the pct_change function of pandas is used for the calculation.
## Volatility
Next, we calculate the historical volatility. The historical volatility calculated here is the standard deviation of the logarithmic price change over the last 10, 21, and 63 business days. Historical volatility is a risk indicator and is used to determine how violently prices have fluctuated. In general, stocks with large historical volatility are considered relatively riskier to hold as assets than stocks with small historical volatility.
## Moving Average
A moving average (MA) is a widely used technical indicator that smooths out price trends by filtering out the noise from random short-term price fluctuations.Moving averages can be constructed in several different ways and employ different numbers of days for the averaging interval.The most common applications of moving averages are to identify trend direction and to determine support and resistance levels.When asset prices cross over their moving averages, it may generate a trading signal for technical traders.
## Value At Risk
Value at Risk (VaR) is a statistic used to try and quantify the level of financial risk within a firm or portfolio over a specified time frame. VaR provides an estimate of the maximum loss from a given position or portfolio over a period of time, and you can calculate it across various confidence levels. We will be using the variance - covariance method to calculate the VaR of each stock on daily basis.


In [37]:
import scipy
from scipy.stats import norm
from tabulate import tabulate

In [38]:
def get_features_for_predict(price, code):
    """
    Args:
        price (pd.DataFrame)  : pd.DataFrame include stock_price
        code (int)  : A local code for a listed company
    Returns:
        feature DataFrame (pd.DataFrame)
    """
    close_col = "AdjustedClose"
    feats = price.loc[price["SecuritiesCode"] == code, ["SecuritiesCode", close_col]].copy()

    # calculate 2 week return using AdjustedClose
    feats["return_2week"] = feats[close_col].pct_change(10)
    # calculate last 1 month return using AdjustedClose
    feats["return_1month"] = feats[close_col].pct_change(21)
    # calculate last 3 months return using AdjustedClose
    feats["return_3month"] = feats[close_col].pct_change(63)
    
    # calculate 2 week historical volatility using AdjustedClose
    feats["volatility_2week"] = (
        np.log(feats[close_col]).diff().rolling(10).std()
    )
    # calculate last 1 month historical volatility using AdjustedClose
    feats["volatility_1month"] = (
        np.log(feats[close_col]).diff().rolling(21).std()
    )
    # calculate last 3 months historical volatility using AdjustedClose
    feats["volatility_3month"] = (
        np.log(feats[close_col]).diff().rolling(63).std()
    )

    # calculate 2 week moving average mean of return using AdjustedClose 
    feats["MA_2week"] = feats[close_col].pct_change().rolling(10).mean()
    # calculate 1 month moving average mean of return using AdjustedClose 
    feats["MA_1month"] = feats[close_col].pct_change().rolling(21).mean()
    # calculate 3 months moving average mean of return using AdjustedClose 
    feats["MA_3month"] = feats[close_col].pct_change().rolling(63).mean()
    
    # calculate last 2 week Value at Risk using AdjustedClose 
    feats["VaR_95%_2week"] = norm.ppf(0.05, feats["MA_2week"], feats["volatility_2week"])
    # calculate last 1 month Value at Risk using AdjustedClose
    feats["VaR_95%_1month"] = norm.ppf(0.05, feats["MA_1month"], feats["volatility_1month"])
    # calculate last 3 months Value at Risk using AdjustedClose
    feats["VaR_95%_3month"] = norm.ppf(0.05, feats["MA_3month"], feats["volatility_3month"])
                                                   
    # filling data for nan and inf
    feats = feats.fillna(0)
    feats = feats.replace([np.inf, -np.inf], 0)
    # drop AdjustedClose column
    feats = feats.drop([close_col], axis=1)
    

    return feats

In [39]:
# fetch prediction target SecuritiesCodes
codes = sorted(merged_df["SecuritiesCode"].unique())
len(codes)

2000

In [40]:
# generate feature
buff = []
for code in tqdm(codes):
    feat = get_features_for_predict(merged_df, code)
    buff.append(feat)
feature = pd.concat(buff)

  2%|█▏                                                                              | 31/2000 [00:00<00:47, 41.38it/s]C:\Users\Tram\anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:2128: RuntimeWarning: invalid value encountered in multiply
  lower_bound = _a * scale + loc
C:\Users\Tram\anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:2129: RuntimeWarning: invalid value encountered in multiply
  upper_bound = _b * scale + loc
100%|██████████████████████████████████████████████████████████████████████████████| 2000/2000 [00:44<00:00, 44.61it/s]


In [41]:
feature.tail()

,SecuritiesCode,return_2week,return_1month,return_3month,volatility_2week,volatility_1month,volatility_3month,MA_2week,MA_1month,MA_3month,VaR_95%_2week,VaR_95%_1month,VaR_95%_3month
Date,,,,,,,,,,,,,
2021-11-29,9997,-0.116402,-0.170186,-0.176326,0.012748,0.012622,0.013956,-0.012227,-0.008769,-0.002979,-0.033196,-0.029530,-0.025934
2021-11-30,9997,-0.107095,-0.171429,-0.176543,0.013207,0.012573,0.013955,-0.011186,-0.008840,-0.002983,-0.032909,-0.029521,-0.025938
2021-12-01,9997,-0.062927,-0.141604,-0.171705,0.017227,0.014779,0.014135,-0.006345,-0.007141,-0.002888,-0.034681,-0.031450,-0.026138
2021-12-02,9997,-0.068120,-0.117419,-0.167883,0.016934,0.013932,0.014131,-0.006902,-0.005838,-0.002815,-0.034756,-0.028755,-0.026059
2021-12-03,9997,-0.027933,-0.085414,-0.163462,0.017270,0.014519,0.014237,-0.002695,-0.004143,-0.002730,-0.031102,-0.028023,-0.026148


In [42]:

# calculate periodic return:
merged_df['Daily_return'] = merged_df['AdjustedClose'].pct_change(1)

# Calculate mean and standard deviation of periodic returns:
mean = np.mean(merged_df['Daily_return'])
std_dev = np.std(merged_df['Daily_return'])

#Calculate VaR using point percentile function
VaR_95 = norm.ppf(0.05, mean, std_dev)
VaR_99 = norm.ppf(0.01, mean, std_dev)
print (tabulate([['95%', VaR_95], ['99%', VaR_99]], headers = ['Confidence Level', 'Value at Risk']))


Confidence Level      Value at Risk
------------------  ---------------
95%                       -0.166449
99%                       -0.23583


Meaning that at 95% confidence, the maximum loss incurrs over the whole timeline of the portfolio is likely to be 16.65%. Likewise, at 99% confidence, the maximum loss possible is 23.58%. 

In [43]:
feature.isnull().sum()

SecuritiesCode       0
return_2week         0
return_1month        0
return_3month        0
volatility_2week     0
volatility_1month    0
volatility_3month    0
MA_2week             0
MA_1month            0
MA_3month            0
VaR_95%_2week        0
VaR_95%_1month       0
VaR_95%_3month       0
dtype: int64

In [44]:
feature = (pd.concat([feature, merged_df], axis=1)).drop(['SecuritiesCode','Target', 'EndDate', 'Daily_return'], axis=1)
feature = pd.concat([merged_df['SecuritiesCode'], feature], axis=1)
feature = feature.fillna(0) 
feature

,SecuritiesCode,return_2week,return_1month,return_3month,volatility_2week,volatility_1month,volatility_3month,MA_2week,MA_1month,MA_3month,VaR_95%_2week,VaR_95%_1month,VaR_95%_3month,Volume,ExpectedDividend,AdjustedClose,ForecastEarningsPerShare,TotalSales,TotalPurchases
Date,,,,,,,,,,,,,,,,,,,
2017-01-04,1301,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,31400,0.0,2742.0,0.0,3.052787e+09,3.054814e+09
2017-01-05,1301,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,17900,0.0,2738.0,0.0,3.052787e+09,3.054814e+09
2017-01-06,1301,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,19900,0.0,2740.0,0.0,3.052787e+09,3.054814e+09
2017-01-10,1301,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,24200,0.0,2748.0,0.0,3.710609e+09,3.709987e+09
2017-01-11,1301,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,9300,0.0,2745.0,0.0,3.710609e+09,3.709987e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-11-29,9997,-0.116402,-0.170186,-0.176326,0.012748,0.012622,0.013956,-0.012227,-0.008769,-0.002979,-0.033196,-0.029530,-0.025934,320800,0.0,668.0,129.3,4.211296e+09,4.209213e+09
2021-11-30,9997,-0.107095,-0.171429,-0.176543,0.013207,0.012573,0.013955,-0.011186,-0.008840,-0.002983,-0.032909,-0.029521,-0.025938,296300,0.0,667.0,129.3,4.211296e+09,4.209213e+09
2021-12-01,9997,-0.062927,-0.141604,-0.171705,0.017227,0.014779,0.014135,-0.006345,-0.007141,-0.002888,-0.034681,-0.031450,-0.026138,339100,0.0,685.0,129.3,4.211296e+09,4.209213e+09


In [45]:
# Creating label
def get_label(price, code):
    """ Labelizer
    Args:
        price (pd.DataFrame): dataframe of stock_price.csv
        code (int): Local Code in the universe
    Returns:
        df (pd.DataFrame): label data
    """
    df = price.loc[price["SecuritiesCode"] == code].copy()
    df.loc[:, "label"] = df["Target"]

    return df.loc[:, ["SecuritiesCode", "label"]]

In [46]:
# split data into TRAIN and TEST
TRAIN_END = "2019-12-31"
# We put a week gap between TRAIN_END and TEST_START
# to avoid leakage of test data information from label
TEST_START = "2020-01-06"

def get_features_and_label(price, codes, features):
    """
    Args:
        price (pd.DataFrame): loaded price data
        codes  (array) : target codes
        feature (pd.DataFrame): features
    Returns:
        train_X (pd.DataFrame): training data
        train_y (pd.DataFrame): label for train_X
        test_X (pd.DataFrame): test data
        test_y (pd.DataFrame): label for test_X
    """
    # to store splited data
    trains_X, tests_X = [], []
    trains_y, tests_y = [], []

    # generate feature one by one
    for code in tqdm(codes):

        feats = features[features["SecuritiesCode"] == code].dropna()
        labels = get_label(price, code).dropna()

        if feats.shape[0] > 0 and labels.shape[0] > 0:
            # align label and feature indexes
            labels = labels.loc[labels.index.isin(feats.index)]
            feats = feats.loc[feats.index.isin(labels.index)]

            assert (labels.loc[:, "SecuritiesCode"] == feats.loc[:, "SecuritiesCode"]).all()
            labels = labels.loc[:, "label"]

            # split data into TRAIN and TEST
            _train_X = feats[: TRAIN_END]
            _test_X = feats[TEST_START:]

            _train_y = labels[: TRAIN_END]
            _test_y = labels[TEST_START:]
            
            assert len(_train_X) == len(_train_y)
            assert len(_test_X) == len(_test_y)

            # store features
            trains_X.append(_train_X)
            tests_X.append(_test_X)
            # store labels
            trains_y.append(_train_y)
            tests_y.append(_test_y)
            
    # combine features for each codes
    train_X = pd.concat(trains_X)
    test_X = pd.concat(tests_X)
    # combine label for each codes
    train_y = pd.concat(trains_y)
    test_y = pd.concat(tests_y)

    return train_X, train_y, test_X, test_y

In [47]:
# generate feature/label
train_X, train_y, test_X, test_y = get_features_and_label(
    merged_df, codes, feature
)

100%|██████████████████████████████████████████████████████████████████████████████| 2000/2000 [00:55<00:00, 36.11it/s]


# Building LightGBM (Light Gradient Boosting Machine) Model

In [65]:
lgbm_params = {
    'seed': 42,
    'n_jobs': -1,
}

feat_cols = [
    "return_2week",
    "return_1month",
    "return_3month",
    "volatility_2week",
    "volatility_1month",
    "volatility_3month",
    "MA_2week",
    "MA_1month", 
    "MA_3month", 
    "VaR_95%_2week", 
    "VaR_95%_1month",
    "VaR_95%_3month", 
    "Volume", 
    "ExpectedDividend", 
    "AdjustedClose",
    "ForecastEarningsPerShare", 
    "TotalSales", 
    "TotalPurchases"
]


In [49]:
pip install lightgbm

Note: you may need to restart the kernel to use updated packages.


In [66]:
from lightgbm import LGBMRegressor
# initialize model
pred_model = LGBMRegressor(**lgbm_params)
# train
pred_model.fit(train_X[feat_cols].values, train_y)
# prepare result data
result = test_X[["SecuritiesCode"]].copy()
# predict
result.loc[:, "predict"] = pred_model.predict(test_X[feat_cols])
# actual result
result.loc[:, "Target"] = test_y.values

def set_rank(df):
    """
    Args:
        df (pd.DataFrame): including predict column
    Returns:
        df (pd.DataFrame): df with Rank
    """
    # sort records to set Rank
    df = df.sort_values("predict", ascending=False)
    # set Rank starting from 0
    df.loc[:, "Rank"] = np.arange(len(df["predict"]))
    return df

result = result.sort_values(["Date", "predict"], ascending=[True, False])
result = result.groupby("Date").apply(set_rank)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.078267 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1530
[LightGBM] [Info] Number of data points in the train set: 1401161, number of used features: 6
[LightGBM] [Info] Start training from score 0.000469


In [68]:
result.tail()

,SecuritiesCode,predict,Target,Rank
Date,,,,
2021-12-03,6629,-0.003969,0.039900,1995
2021-12-03,9107,-0.004079,0.038168,1996
2021-12-03,7777,-0.004380,-0.020528,1997
2021-12-03,4113,-0.007303,0.033153,1998
2021-12-03,7047,-0.008192,0.055768,1999


## Evaluation

In [53]:
def calc_spread_return_sharpe(df: pd.DataFrame, portfolio_size: int = 200, toprank_weight_ratio: float = 2) -> float:
    """
    Args:
        df (pd.DataFrame): predicted results
        portfolio_size (int): # of equities to buy/sell
        toprank_weight_ratio (float): the relative weight of the most highly ranked stock compared to the least.
    Returns:
        (float): sharpe ratio
    """
    def _calc_spread_return_per_day(df, portfolio_size, toprank_weight_ratio):
        """
        Args:
            df (pd.DataFrame): predicted results
            portfolio_size (int): # of equities to buy/sell
            toprank_weight_ratio (float): the relative weight of the most highly ranked stock compared to the least.
        Returns:
            (float): spread return
        """
        assert df['Rank'].min() == 0
        assert df['Rank'].max() == len(df['Rank']) - 1
        weights = np.linspace(start=toprank_weight_ratio, stop=1, num=portfolio_size)
        purchase = (df.sort_values(by='Rank')['Target'][:portfolio_size] * weights).sum() / weights.mean()
        short = (df.sort_values(by='Rank', ascending=False)['Target'][:portfolio_size] * weights).sum() / weights.mean()
        return purchase - short

    buf = df.groupby('Date').apply(_calc_spread_return_per_day, portfolio_size, toprank_weight_ratio)
    sharpe_ratio = buf.mean() / buf.std()
    return sharpe_ratio

In [54]:
calc_spread_return_sharpe(result, portfolio_size=200)

0.09670030335485494

In [70]:
pred_model.booster_.save_model("LGBM-model.txt")